## **Name: Floyd Mabiala"**
## **Implementation of A Dynamic Evolving Neural Fuzzy Inference System**

### Implementation of a DENFIS steps:

  a. Build the Framework

  b. Define & compute Gaussian member Function

  c. Initial Predition using TSK

  d. Online training
  -  Rule creation (center, sigma, Consequence = [y_t, 0, 0 __init__]
  -  Compute the distance (Euclidian distance)
  -  Novelty detection (Threshold-based Method)
  -  Update New rules: update (Center, sigma, consequence)

  e. Prediction using K-nearest rules
  
  - Compute the didstance
  - Compute the Activation
  - Weighted average

  f. Compute the Model prediction on Test Dataset
  
  g. Evaluate the Model performance

##References

- Lughofer, Edwin. Evolving fuzzy systems-methodologies, advanced concepts and applications. Vol. 53. Berlin: Springer, 2011.

- Kisi, Ozgur, Iman Mansouri, and Jong Wan Hu. "A new method for evaporation modeling: Dynamic evolving neural‐fuzzy inference system." Advances in Meteorology 2017.1 (2017): 5356324.

- Lughofer, Edwin, et al. "Generalized smart evolving fuzzy systems." Evolving systems 6.4 (2015): 269-292.

- Stock Trading: K. K. Ang and C. Quek, "Stock Trading Using RSPOP: A Novel Rough Set-Based Neuro-Fuzzy Approach," in IEEE Transactions on Neural Networks, vol. 17, no. 5, pp. 1301-1315, Sept. 2006, doi: 10.1109/TNN.2006.875996.
- DENFIS: N. K. Kasabov and Qun Song, "DENFIS: dynamic evolving neural-fuzzy inference system and its application for time-series prediction," in IEEE Transactions on Fuzzy Systems, vol. 10, no. 2, pp. 144-154, April 2002, doi: 10.1109/91.995117.

##**DENFIS**
In this project, **DENFIS** is the model that will be used to predict the Power Output of a Renewable Energy Source. DENFIS, or Dynamic Evolving Neural-Fuzzy Inference System, can be broadly characterized as an extension of ANFIS (Adaptive Neuro-Fuzzy Inference System) with the added capability of automatically generating membership functions. More formally, DENFIS is defined as an adaptive and intelligent computational framework designed to handle complex tasks such as time series prediction. Its distinguishing feature lies in its ability to construct and continuously refine a fuzzy inference system in real time, dynamically adjusting its structure and parameters based on the data encountered during the learning phase. This evolving nature enables DENFIS to respond effectively to non-stationary environments and incremental data streams, making it particularly suitable for applications requiring high adaptability and precision [Kasabov, 2002](https://ieeexplore.ieee.org/document/995117)


## Dynamic Evolving Neural Fuzzy Inference System


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# DENFIS
# Parameters: Maximum number of rules (m_max), distance threshold (theta), Number of nearest-neighbors (k), learning rate (lr), Sigma

class DENFIS:
    def __init__(self,
                 m_max=60,
                 distance_threshold=0.25,
                 K=4,
                 lr=0.06,
                 init_sigma=0.3):

        self.m_max = m_max
        self.theta = distance_threshold
        self.K = K
        self.lr = lr
        self.init_sigma = init_sigma

        self.centers = []       # rule centers
        self.sigmas = []        # widths
        self.consequents = []   # linear TSK parameters: [w0, w1, w2] for y = w0 + w1*G + w2*T

    # ---------------------------------------
    # Gaussian membership: fn (x, c, sigma)
    # ---------------------------------------
    def gaussian(self, x, center, sigma):
        return np.exp(-np.sum((x - center)**2) / (2 * sigma**2))

    # ---------------------------------------==-------------------------------
    # Takagi-Sugeno-Kang (TSK) rule output: here the consequence is a linear equation
    # y = w_0 + w_1 x [0] + w_2 x [1] + ... + w_n x [n]
    # The Conseq part of the TSK rule approximates the function locally around the C
    # ---------------------------------------==--------------------------------
    def tsk_output(self, x, params):
        w0, w1, w2 = params
        return w0 + w1 * x[0] + w2 * x[1]

    # ---------------------------------------==-------------------------------
    # Online training step. it handels both structure evolution (adding rules), &
    # params updates
    # ---------------------------------------==-------------------------------
    def train(self, x_t, y_t):
        x_t = np.array(x_t)

        # -------------------------------
        # If no rules → create first rule
        # -------------------------------
        if len(self.centers) == 0:
            self.centers.append(x_t.copy())
            self.sigmas.append(self.init_sigma)
            # TSK params: bias, G coeff, T coeff
            self.consequents.append(np.array([y_t, 0, 0], dtype=float))

            # return y_t as a prediction for the first point
            return y_t

        # -------------------------------==-------------------------------------
        # Compute distances to rule centers: Calculates the Euclidean disatnce x_t
        # to all existing centers, & find the closest winning rule:
        # index (i_best) - & - distance (d_best)
        # -------------------------------==-------------------------------------
        distances = [np.linalg.norm(x_t - c) for c in self.centers]
        i_best = np.argmin(distances)
        d_best = distances[i_best]

        # -------------------------------==-----------------------------------
        # Novelty detection & rule creation → new rule: Threshold-based Method
        # -------------------------------==-----------------------------------

        # if d_best>theta (novel pt) & rules<ma_max
        if (d_best > self.theta) and (len(self.centers) < self.m_max):

            self.centers.append(x_t.copy())
            self.sigmas.append(self.init_sigma)
            self.consequents.append(np.array([y_t, 0, 0], dtype=float))

            return y_t

        # -------------------------------==---------------
        # Update the winning rule (if no new rules added)
        # -------------------------------==---------------
        c = self.centers[i_best]
        s = self.sigmas[i_best]
        w = self.consequents[i_best]

        # Update center, x_t: new_center = old_center + lr * (x_t - old_center)
        self.centers[i_best] = c + self.lr * (x_t - c)

        # Update width (sigma): new_sigma = old_sigma + lr * (mean(|x_t-center|)-old_sigma)
        self.sigmas[i_best] = s + self.lr * (np.abs(x_t - c).mean() - s)

        # Update consequent (gradient descent)
        #compute the output y_pred using TSK_ouput
        y_pred_r = self.tsk_output(x_t, w)

        error = y_t - y_pred_r

        grad = np.array([1, x_t[0], x_t[1]])  # derivative with r.to TSK parameters


        # Incremental learning (update):
        # new_param = old_param + lr*err*grad (GD for LS minization)
        self.consequents[i_best] = w + self.lr * error * grad

        # -----------------------------------------
        # Prediction using DENFIS (K-nearest rules)
        # -----------------------------------------
        return self.predict(x_t)

    # ---------------------------------------==-----------------------
    # DENFIS prediction: K-nearest Rules, only the most relevant rules
    # ---------------------------------------==-----------------------
    def predict(self, x):
        if len(self.centers) == 0:
            return 0

        x = np.array(x)

        # Find K-nearest Rules: Compute distances
        distances = np.array([np.linalg.norm(x - c) for c in self.centers])
        idx = np.argsort(distances)[:self.K]

        # Compute activations and weighted output
        activations = np.array([self.gaussian(x, self.centers[i], self.sigmas[i])
                                for i in idx]) # For each rule: active=gauss(x,c,s)

        outputs = np.array([self.tsk_output(x, self.consequents[i])
                            for i in idx]) # rule_output = tsk_output(x, conseq)

        # Defuzzification
        if np.sum(activations) == 0:
            return outputs.mean()

        return np.sum(activations * outputs) / np.sum(activations) #weighted average


##For PV Modeling:
### Training DENFIS using PV data such as irradiance (G) & temperature (T) --> ref voltage Vref

###Power Prediction Using DENFIS
###A simulation of PV voltage control using DENFIS as power prediction model.

In [ ]:
# Instatiating DENFIS

denfis = DENFIS(
    m_max=60,
    distance_threshold=0.3,
    K=4,
    lr=0.055
)

# Simulated PV dataset
for G, T, Vref in #dataset:   # dataset = list of (G, T, Vref)
    denfis.train([G, T], Vref)


##Make a Prediction for G and T

In [7]:
G_new = 900     # W/m^2
T_new = 35      # °C

Vref_pred = denfis.predict([G_new, T_new])

print("Predicted Reference Voltage:", Vref_pred)


Predicted Reference Voltage: 0
